[this doc on github](https://github.com/dotnet/interactive/tree/master/samples/notebooks/fsharp/Docs)

# Displaying Output in an F# notebook

This article describes how to display output results in F# notebooks.

When writing F# in a .NET notebook, you can write code similar to how you would with the F# Interactive tool.

In [ ]:
1 + 3

The last value in a cell has its output displayed. When you end a cell with an expression evaluation like this, it is the return value of the cell. There can only be a single return value for a cell. If you add more code after a return value expression, you'll see a compile error. 

Here is another example:

In [ ]:
let r = System.Random()
r.Next(0,10)

You can also evaluate a string:

In [ ]:
"Hello, world!"


### Using Console Output

You can display information without using the return value. The most intuitive way is to write to the console:

In [ ]:
System.Console.WriteLine("Hello, world!")
printfn "Hello, world!"

You can also use `%A` printing to print structured values:

In [ ]:
printfn "%120A" [ for i in 1 .. 30 -> [ 1 .. i ]]

## Using `display`

A more familiar API for many notebook users would be the `display` method.

In [ ]:
display("Hello, world!")

Many outputs are automatically formatted as HTML by `display` or output-value formatting.


### Updating 'display' results asynchronously

Each call to `display` writes an additional display data value to the notebook.
You can also update an existing displayed value by calling `Update` on the object returned by a call to `display`.

In some environments such as Jupyter Lab, only updates made during the execution of the cell are applied.

In [ ]:
let fruitOutput = display("Let's get some fruit!");
let basket = [| "apple"; "orange"; "coconut"; "pear"; "peach" |]

for fruit in basket do
    System.Threading.Thread.Sleep(1000);
    let updateMessage = sprintf "I have 1 %s" fruit
    fruitOutput.Update(updateMessage)

System.Threading.Thread.Sleep(1000);

fruitOutput.Update(basket);

## Displaying HTML



To display arbitrary HTML, the helper function `HTML` can be used.


In [ ]:
HTML(Html.raw "<b style=\"color:blue\">Hello!</b>")

HTML can be specified using any DSL that generates ASP.NET Core `IHtmlContent`.  The `Html` module contains an F# DSL for HTML specification similar to [Giraffe.ViewEngine](https://github.com/giraffe-fsharp/Giraffe.ViewEngine#html-elements-and-attributes) that does this. For example:


In [ ]:
open Html

let indexView =
    div [] [
      h4 [] [ str "I love F#" ]
      p [ _class "some-css-class"; _id "someId" ] [
          b  [] [str "Hello" ]
          str " "
          i [] [str "World"]
      ]
    ]

display(indexView)

## Executing Javascript

The helper function `Javascript` executes Javascript directly.

In [ ]:
Javascript(@"alert(""Hello!"");")

## Defining CSS

The `CSS` function can be used to add CSS styling to the host HTML system. Here are some examples:

In [ ]:
CSS "h2 { color: darkslategrey; }"

## HTML Formatting 



When you return a value or a display a value in a .NET notebook, the default formatting behavior is to try to provide some useful information about the object. If it's an array or other type implementing `IEnumerable`, that might look like this:

In [ ]:
display ["hello"; "world"]

[ 1 .. 4 ]

As you can see, by default a table structure is used whether you pass the object to the `display` method or return it as the cell's value.


### Switching to plain text only

If you prefer, you can switch to plain text by default.  The simplest way to do this is to register `"text/plain"` as the default mime type for all objects and switch the `%A` printing.  

In [ ]:
Formatter.SetPreferredMimeTypeFor(typeof<obj>, "text/plain")
Formatter.Register(fun (x:obj) (writer: TextWriter) -> fprintfn writer "%120A" x )

In [ ]:
[ 1 .. 4 ]

Now reset back to defaults:

In [ ]:
Formatter.ResetToDefault()

### Preserving multi-line string outputs

By default multi-line content does not have formatting preserved.  For example:

In [ ]:
[ sprintf "%120A" [ for i in 1 .. 30 -> [ 1 .. i ]] ]

You can use styling to preserve the preformatting of such content.

In [ ]:
CSS ".dni-plaintext { text-align: left; white-space: pre; font-family: monospace; });"

Note that `Formatter.ResetToDefault()` doesn't undo any CSS changes. You can undo this change with:

In [ ]:
CSS ".dni-plaintext { text-align: inherit; white-space: inherit; font-family: inherit; });"

### HTML Formatting of objects

The default formatting behavior for many objects is to produce a table showing their properties and the values of those properties.

In [ ]:
type Person = { FirstName: string; LastName: string; Age: int }

// Evaluate a new person
{ FirstName = "Mitch"; LastName = "Buchannon"; Age = 42 }

###  HTML Formatting of sequences

When you have a collection of objects, you can see the values listed for each item in the collection:

In [ ]:
let people =
    [
        { FirstName = "Mitch"; LastName = "Buchannon"; Age = 42 }
        { FirstName = "Hobie "; LastName = "Buchannon"; Age = 23 }
        { FirstName = "Summer"; LastName = "Quinn"; Age = 25 }
        { FirstName = "C.J."; LastName = "Parker"; Age = 23 }
    ]

people

### HTML Formatting of dictionaries

Similarly to the behavior for `IEnumerable` objects, you'll also see table output for dictionaries, but for each value in the dictionary, the key is provided rather than the index within the collection.

In [ ]:
// Cannot simply use 'dict' here, see https://github.com/dotnet/interactive/issues/12

let d = dict [("zero", 0); ("one", 1); ("two", 2)]
d

###  HTML Formatting of nested objects

Now let's try something a bit more complex. Let's look at a graph of objects. 

We'll redefine the `Person` class to allow a reference to a collection of other `Person` instances.

In [ ]:
type Person =
    { FirstName: string
      LastName: string
      Age: int
      Friends: ResizeArray<Person> }

let mitch = { FirstName = "Mitch"; LastName = "Buchannon"; Age = 42; Friends = ResizeArray() }
let hobie = { FirstName = "Hobie "; LastName = "Buchannon"; Age = 23; Friends = ResizeArray() }
let summer =  { FirstName = "Summer"; LastName = "Quinn"; Age = 25; Friends = ResizeArray() }

mitch.Friends.AddRange([ hobie; summer ])
hobie.Friends.AddRange([ mitch; summer ])
summer.Friends.AddRange([ mitch; hobie ])

let people = [ mitch; hobie; summer ]
display people

That's a bit hard to read, right?  The defaut formatting behaviors are not always as useful as they might be. In order to give you more control in these kinds of cases, formatters can be customized from within the .NET notebook.

### Registering custom plain text formatters

Let's clean up the output above by customizing the formatter for the `Person.Friends` property, which is creating a lot of noise. 

The way to do this is to use the `Formatter` API. This API lets you customize the formatting for a specific type. Since `Person.Friends` is a sequence of `Person` objects, i.e. type `seq<Person>`, we can register a custom formatter for that type to change the output. Let's just list their first names:

In [ ]:
Formatter.Register<seq<Person>>(
    mimeType = "text/plain",
    formatter = Func<_,_,_,_>(fun context people (writer: TextWriter) ->
        for person in people do
            writer.Write(person.FirstName)
            writer.Write(" ")
        true))


Now display the `people` data again:

In [ ]:
people

You might have noticed that `people` is of type `ResizeArray<Person>`, but the table output still includes columns for `LastName`, `Age`, and `Friends`. What's going on here?

Notice that the custom formatter we just registered was registered for the mime type `"text/plain"`. The top-level formatter that's used when we call `display` requests output of mime type `"text/html"` and the nested objects are formatted using `"text/plain"`. It's the nested objects, not the top-level HTML table, that's using the custom formatter here.


### Registering custom HTML formatters

The HTML formatters apply by default to many objects. For example, to replace the default HTML table view for a particular collection type, you can register a formatter for the `"text/html"` mime type. HTML is specified using the HTML DSL similar to Giraffe's ViewEngine. 

In this case we put the formatter registration in a module to prevent opening the `Html` DSL everywhere.

In [ ]:
module PersonHtmlFormatter = 
    
    // Locally open the F# HTML DSL.
    open Html

    Formatter.Register<seq<Person>>(
        mimeType = "text/html",
        formatter = Func<_,_,_,_>(fun (context: FormatContext) (people: seq<Person>) (writer: TextWriter) ->
            table [] [
              thead [ _style ["color: blue"]] [
                th [] [ str "First Name" ]
                th [] [ str "Last Name" ]
                th [] [ str "Age" ]
              ]
              tbody [_style ["color: darkolivegreen"]] [
                for p in people do
                  tr [] [
                    td [] [ str p.FirstName ]
                    td [] [ str p.LastName ]
                    td [] [ str (string p.Age) ; str " years" ] 
                  ]
              ]
            ]
            |> writer.Write
            true))


Now display the `people` data again:

In [ ]:
people